In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [27]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-03.parquet')

In [29]:
df['ride_id'] = f'{2022:04d}/{2:02d}_' + df.index.astype('str')

In [30]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [32]:
print(f"Std Prediction: {y_pred.std()}, mean: {y_pred.mean()}")

Std Prediction: 5.556602554785001, mean: 12.758556818790902


In [23]:
df_result = pd.DataFrame({'ride_id':df['ride_id'].values, 
                        'prediction': y_pred})

In [26]:
df_result.to_parquet(
    'result.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)